In [63]:
import numpy as np
import matplotlib.pyplot
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
file_path = "archive.zip"
data = pd.read_csv(file_path, skiprows=1, delim_whitespace=True, names=["Date","tOpen","tHigh","tLow","tClose","tAdj Close","tVolume","tName"])

In [64]:
data

,Date,tOpen,tHigh,tLow,tClose,tAdj Close,tVolume,tName
0,2010-01-04,7.622500,7.660714,7.585000,7.643214,6.562591,493729600,AAPL
1,2010-01-05,7.664286,7.699643,7.616071,7.656429,6.573935,601904800,AAPL
2,2010-01-06,7.656429,7.686786,7.526786,7.534643,6.469369,552160000,AAPL
3,2010-01-07,7.562500,7.571429,7.466071,7.520714,6.457407,477131200,AAPL
4,2010-01-08,7.510714,7.571429,7.466429,7.570714,6.500339,447610800,AAPL
...,...,...,...,...,...,...,...,...
271675,2021-09-03,295.325012,301.804993,292.029999,298.290009,298.290009,6127900,ZM
271676,2021-09-07,298.295013,300.980011,294.799988,299.959991,299.959991,4251900,ZM
271677,2021-09-08,299.549988,299.959991,290.529999,293.600006,293.600006,3934400,ZM
271678,2021-09-09,292.160004,297.570007,291.130005,295.859985,295.859985,3350100,ZM


In [48]:
print(type(data))

<class 'pandas.core.frame.DataFrame'>


In [49]:
data = data.iloc[:,1:2]

In [50]:
from sklearn.preprocessing import MinMaxScaler

sc = MinMaxScaler()

data_df = sc.fit_transform(data)

In [52]:
X_train = data_df[0:271680]
y_train = data_df[0:271680]

In [55]:
X_train = np.reshape(X_train,(271680,1,1))

In [56]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

2023-01-23 16:46:35.411634: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [58]:
regressor = Sequential()
regressor.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None,1)))

In [60]:
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [61]:
regressor.fit(X_train, y_train, batch_size =32, epochs =200)

Epoch 1/200
8490/8490 [==============================] - 12s 1ms/step - loss: 0.0097
Epoch 2/200
8490/8490 [==============================] - 12s 1ms/step - loss: 0.0012
Epoch 3/200
8490/8490 [==============================] - 13s 1ms/step - loss: 4.1862e-04
Epoch 4/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.7838e-04
Epoch 5/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.5523e-04
Epoch 6/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.3812e-04
Epoch 7/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.2576e-04
Epoch 8/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.1672e-04
Epoch 9/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.0996e-04
Epoch 10/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.0491e-04
Epoch 11/200
8490/8490 [==============================] - 11s 1ms/step - loss: 3.0087e-04
Epoch 12/200
8490/8490 [===

KeyboardInterrupt: 